In [2]:
from llama_index.core import (
    VectorStoreIndex
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

/root/miniconda3/envs/784/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from IPython.display import Markdown, display

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

In [5]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [6]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [7]:
from llama_index.core import SQLDatabase
GROQ_API_KEY = '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'

In [8]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [9]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {
        "city_name": "Chicago",
        "population": 2679000,
        "country": "United States",
    },
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [10]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [11]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


In [12]:
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

In [13]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [14]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm, embed_model=embed_model
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)
print(response.metadata['sql_query'])

{'7a57b99c-7040-4542-b511-3b34363d50ac': {'sql_query': 'SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 1;', 'result': [('Tokyo', 13960000)], 'col_keys': ['city_name', 'population']}, 'sql_query': 'SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 1;', 'result': [('Tokyo', 13960000)], 'col_keys': ['city_name', 'population']}


In [15]:
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo, with a population of approximately 13,960,000 people.</b>

In [16]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True,
    llm=llm, embed_model=embed_model
)

In [17]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [18]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

ModuleNotFoundError: No module named 'matplotlib'

In [21]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False, llm=llm, embed_model=embed_model
)

In [22]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [23]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 198178f2-3274-439d-b2bf-d8718579d8b7<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Tokyo', 'population': 13960000}<br>

**Node ID:** c635121f-93a6-448e-993b-0f72f12aaf83<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Seoul', 'population': 9776000}<br>

**Node ID:** b98a3646-f2b5-44d2-84af-773ab6ce9940<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Toronto', 'population': 2930000}<br>

**Node ID:** fcef1681-1792-4f87-8935-953a855181bc<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Chicago', 'population': 2679000}<br>

In [24]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever, llm=llm, embed_model=embed_model)

In [25]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [26]:
print(str(response))

Based on the provided information, the top 5 cities with the highest population are not available since only 4 cities are provided. However, here are the 4 cities in the order of their population:

1. Tokyo - 13960000
2. Seoul - 9776000
3. Toronto - 2930000
4. Chicago - 2679000
